In [1]:
import random
import numpy as np
import tensorflow as tf
import mlflow.keras

from ml_investing_wne import config
from ml_investing_wne.data_engineering.load_data import get_hist_data
from ml_investing_wne.data_engineering.prepare_dataset import prepare_processed_dataset
from ml_investing_wne.train_test_val_split import train_test_val_split
from ml_investing_wne.helper import get_ml_flow_experiment_name, get_callbacks, \
    get_final_model_path, evaluate_model
from ml_investing_wne.models import model_factory
from ml_investing_wne.utils import get_logger
from ml_investing_wne.data_engineering.crypto_factory import CryptoFactory
from ml_investing_wne.data_engineering.prepare_dataset import prepare_processed_dataset
random.seed(config.seed)
np.random.seed(config.seed)
tf.random.set_seed(config.seed)

logger = get_logger()

/Users/i0495036/Documents/sandbox/ml_investing_wne/ml_investing_wne/venv/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None

In [3]:

if config.RUN_TYPE == 'forex':
    if config.provider == 'hist_data':
        df = get_hist_data(currency=config.currency)
    else:
        logger.error('not implemented')
elif config.RUN_TYPE == 'crypto':
    crypto = CryptoFactory(config.provider, config.currency)
    # crypto.generate_volumebars(frequency=2000)
    crypto.time_aggregation(freq='60min')
    # df = crypto.df_time_aggregated
    # df = crypto.df_volume_bars
    crypto.run_3_barriers()
    df = crypto.df_3_barriers
    logger.info(f'df head: {df.head()}')
df = prepare_processed_dataset(df=df, add_target=False)


2022-12-08 22:24:52,184 - ml_investing_wne.data_engineering.prepare_dataset - exported to /Users/i0495036/Documents/sandbox/ml_investing_wne/ml_investing_wne/src/ml_investing_wne/data/processed/EURCHF/EURCHF_processed_60min.csv


In [4]:
df

,open,high,low,close,SMA_3,EMA_3,VAR_3,SMA_5,EMA_5,VAR_5,SMA_10,EMA_10,VAR_10,SMA_13,EMA_13,VAR_13,SMA_20,EMA_20,VAR_20,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,RSI_10,RSI_6,STOCHk_14_3_3,STOCHd_14_3_3,WILLR_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,roc_1,hour,weekday,hour_sin,hour_cos,weekday_sin,weekday_cos
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-05 02:00:00,1.48560,1.48630,1.48440,1.48530,1.485233,1.485131,2.533333e-07,1.484680,1.484831,7.120000e-07,1.484210,1.484500,8.032222e-07,1.484131,1.484410,6.656410e-07,1.484415,1.484311,1.240289e-06,0.000121,0.000190,-0.000069,57.530386,59.356492,64.594559,87.805933,72.897040,-25.641026,1.483171,1.484680,1.486189,0.203335,0.705375,0.999731,2,1,0.519584,0.854419,0.866025,0.5
2010-01-05 03:00:00,1.48520,1.48610,1.48520,1.48540,1.485467,1.485266,4.333333e-08,1.485020,1.485021,4.570000e-07,1.484310,1.484663,9.454444e-07,1.484292,1.484552,7.141026e-07,1.484340,1.484415,9.604211e-07,0.000189,0.000206,-0.000017,58.127895,60.152106,65.805176,82.780292,82.427015,-23.076923,1.483811,1.485020,1.486229,0.162866,0.657116,1.000067,3,1,0.730836,0.682553,0.866025,0.5
2010-01-05 04:00:00,1.48540,1.48560,1.48340,1.48410,1.484933,1.484683,5.233333e-07,1.485040,1.484714,4.080000e-07,1.484290,1.484561,9.498889e-07,1.484292,1.484487,7.141026e-07,1.484205,1.484385,6.257632e-07,0.000136,0.000123,0.000014,48.562654,46.892862,42.914275,64.957265,78.514496,-56.410256,1.483897,1.485040,1.486183,0.153885,0.088668,0.999125,4,1,0.887885,0.460065,0.866025,0.5
2010-01-05 05:00:00,1.48420,1.48500,1.48390,1.48440,1.484633,1.484541,4.633333e-07,1.484980,1.484609,4.770000e-07,1.484320,1.484532,9.462222e-07,1.484308,1.484475,7.141026e-07,1.484150,1.484386,5.363158e-07,0.000117,0.000083,0.000034,50.583556,49.733909,47.930167,57.264957,68.334171,-48.717949,1.483745,1.484980,1.486215,0.166396,0.265273,1.000202,5,1,0.979084,0.203456,0.866025,0.5
2010-01-05 06:00:00,1.48450,1.48620,1.48440,1.48590,1.484800,1.485221,9.300000e-07,1.485020,1.485039,5.570000e-07,1.484630,1.484780,8.601111e-07,1.484423,1.484678,9.102564e-07,1.484240,1.484531,6.888421e-07,0.000220,0.000149,0.000072,59.212352,61.250402,65.904938,61.538462,61.253561,-10.256410,1.483685,1.485020,1.486355,0.179804,0.829572,1.001011,6,1,0.997669,-0.068242,0.866025,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-24 12:00:00,1.03934,1.04009,1.03900,1.03984,1.039567,1.039689,6.413334e-08,1.039744,1.039699,9.423000e-08,1.039896,1.039728,2.269600e-07,1.039716,1.039752,2.875256e-07,1.039673,1.039860,2.858871e-07,-0.000240,0.000069,-0.000309,48.571659,49.986673,52.466168,32.403101,38.036176,-54.883721,1.039195,1.039744,1.040293,0.105627,0.587412,1.000481,12,4,-0.136167,-0.990686,-0.866025,-0.5
2021-12-24 13:00:00,1.03984,1.03989,1.03947,1.03956,1.039580,1.039624,6.280001e-08,1.039670,1.039653,8.720000e-08,1.039776,1.039697,1.405600e-07,1.039753,1.039725,2.543564e-07,1.039665,1.039831,2.863211e-07,-0.000237,0.000058,-0.000295,46.311921,46.693946,46.388076,33.023256,33.901809,-67.906977,1.039142,1.039670,1.040198,0.101617,0.395881,0.999731,13,4,-0.398401,-0.917211,-0.866025,-0.5
2021-12-24 14:00:00,1.03955,1.04010,1.03944,1.03993,1.039777,1.039777,3.723334e-08,1.039638,1.039745,5.872000e-08,1.039746,1.039740,1.192933e-07,1.039818,1.039754,2.158026e-07,1.039625,1.039841,2.286368e-07,-0.000202,0.000074,-0.000276,49.645723,51.394885,54.708243,41.432227,35.619528,-52.912621,1.039205,1.039638,1.040071,0.083390,0.836810,1.000356,14,4,-0.631088,-0.775711,-0.866025,-0.5


In [5]:
crypto = CryptoFactory(config.provider, config.currency, df=df)

In [6]:
crypto.df_time_aggregated.head()

,open,high,low,close,SMA_3,EMA_3,VAR_3,SMA_5,EMA_5,VAR_5,SMA_10,EMA_10,VAR_10,SMA_13,EMA_13,VAR_13,SMA_20,EMA_20,VAR_20,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,RSI_10,RSI_6,STOCHk_14_3_3,STOCHd_14_3_3,WILLR_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,roc_1,hour,weekday,hour_sin,hour_cos,weekday_sin,weekday_cos
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-05 02:00:00,1.4856,1.4863,1.4844,1.4853,1.485233,1.485131,2.533333e-07,1.48468,1.484831,7.120000e-07,1.48421,1.484500,8.032222e-07,1.484131,1.484410,6.656410e-07,1.484415,1.484311,1.240289e-06,0.000121,0.000190,-0.000069,57.530386,59.356492,64.594559,87.805933,72.897040,-25.641026,1.483171,1.48468,1.486189,0.203335,0.705375,0.999731,2,1,0.519584,0.854419,0.866025,0.5
2010-01-05 03:00:00,1.4852,1.4861,1.4852,1.4854,1.485467,1.485266,4.333333e-08,1.48502,1.485021,4.570000e-07,1.48431,1.484663,9.454444e-07,1.484292,1.484552,7.141026e-07,1.484340,1.484415,9.604211e-07,0.000189,0.000206,-0.000017,58.127895,60.152106,65.805176,82.780292,82.427015,-23.076923,1.483811,1.48502,1.486229,0.162866,0.657116,1.000067,3,1,0.730836,0.682553,0.866025,0.5
2010-01-05 04:00:00,1.4854,1.4856,1.4834,1.4841,1.484933,1.484683,5.233333e-07,1.48504,1.484714,4.080000e-07,1.48429,1.484561,9.498889e-07,1.484292,1.484487,7.141026e-07,1.484205,1.484385,6.257632e-07,0.000136,0.000123,0.000014,48.562654,46.892862,42.914275,64.957265,78.514496,-56.410256,1.483897,1.48504,1.486183,0.153885,0.088668,0.999125,4,1,0.887885,0.460065,0.866025,0.5
2010-01-05 05:00:00,1.4842,1.4850,1.4839,1.4844,1.484633,1.484541,4.633333e-07,1.48498,1.484609,4.770000e-07,1.48432,1.484532,9.462222e-07,1.484308,1.484475,7.141026e-07,1.484150,1.484386,5.363158e-07,0.000117,0.000083,0.000034,50.583556,49.733909,47.930167,57.264957,68.334171,-48.717949,1.483745,1.48498,1.486215,0.166396,0.265273,1.000202,5,1,0.979084,0.203456,0.866025,0.5
2010-01-05 06:00:00,1.4845,1.4862,1.4844,1.4859,1.484800,1.485221,9.300000e-07,1.48502,1.485039,5.570000e-07,1.48463,1.484780,8.601111e-07,1.484423,1.484678,9.102564e-07,1.484240,1.484531,6.888421e-07,0.000220,0.000149,0.000072,59.212352,61.250402,65.904938,61.538462,61.253561,-10.256410,1.483685,1.48502,1.486355,0.179804,0.829572,1.001011,6,1,0.997669,-0.068242,0.866025,0.5


In [7]:
crypto.run_3_barriers()

/Users/i0495036/Documents/sandbox/ml_investing_wne/ml_investing_wne/src/ml_investing_wne/data_engineering/crypto_factory.py:187: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for day, vol in daily_volatility.iteritems():
2022-12-08 22:28:29,869 - root - number of ties: 559, share: 0.0075f


In [8]:
crypto.df_3_barriers.head(10)

,open,close,high,low,y_pred
datetime,,,,,
2010-01-05 04:00:00,1.4854,1.4841,1.4856,1.4834,1
2010-01-05 05:00:00,1.4842,1.4844,1.485,1.4839,1
2010-01-05 06:00:00,1.4845,1.4859,1.4862,1.4844,0
2010-01-05 07:00:00,1.4858,1.4856,1.4862,1.4854,0
2010-01-05 08:00:00,1.4856,1.486,1.4862,1.4846,0
2010-01-05 09:00:00,1.4861,1.4859,1.4871,1.4853,0
2010-01-05 10:00:00,1.486,1.4863,1.4874,1.4857,0
2010-01-05 11:00:00,1.4862,1.4866,1.487,1.4857,0
2010-01-05 12:00:00,1.4865,1.4855,1.4866,1.4854,0


In [9]:
crypto.df_3_barriers_additional_info.head(10)

,datetime,prc_change,barrier_touched,barrier_touched_date,top_barrier,bottom_barrier,time_step
0,2010-01-05 04:00:00,0.001333,top,2010-01-05 06:00:00,1.486078,1.482122,2
1,2010-01-05 05:00:00,0.001183,top,2010-01-05 06:00:00,1.486155,1.482645,1
2,2010-01-05 06:00:00,-0.000808,vertical,2010-01-05 17:00:00,1.488243,1.483557,11
3,2010-01-05 07:00:00,-0.000741,vertical,2010-01-05 18:00:00,1.487639,1.483561,11
4,2010-01-05 08:00:00,-0.001209,bottom,2010-01-05 19:00:00,1.487796,1.484204,11
5,2010-01-05 09:00:00,-0.001084,bottom,2010-01-05 19:00:00,1.487511,1.484289,10
6,2010-01-05 10:00:00,-0.000989,bottom,2010-01-05 15:00:00,1.48777,1.48483,5
7,2010-01-05 11:00:00,-0.000902,bottom,2010-01-05 14:00:00,1.487941,1.485259,3
8,2010-01-05 12:00:00,-0.001054,bottom,2010-01-05 19:00:00,1.487066,1.483934,7
9,2010-01-05 13:00:00,-0.001034,bottom,2010-01-05 16:00:00,1.487736,1.484664,3


In [10]:
crypto.df_3_barriers_additional_info.time_step.value_counts(normalize=True)

11    0.191145
2     0.158580
1     0.157573
3     0.119234
4     0.092336
5     0.071710
6     0.057327
7     0.048303
8     0.040152
9     0.034109
10    0.029530
Name: time_step, dtype: float64

In [11]:
print(df.shape)
print(crypto.df_3_barriers.shape)


(74480, 40)
(74478, 5)


In [12]:
df = df.merge(crypto.df_3_barriers[['y_pred']], left_index=True, right_index=True, how='inner')

In [13]:
df

,open,high,low,close,SMA_3,EMA_3,VAR_3,SMA_5,EMA_5,VAR_5,SMA_10,EMA_10,VAR_10,SMA_13,EMA_13,VAR_13,SMA_20,EMA_20,VAR_20,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,RSI_10,RSI_6,STOCHk_14_3_3,STOCHd_14_3_3,WILLR_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,roc_1,hour,weekday,hour_sin,hour_cos,weekday_sin,weekday_cos,y_pred
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-05 04:00:00,1.48540,1.48560,1.48340,1.48410,1.484933,1.484683,5.233333e-07,1.485040,1.484714,4.080000e-07,1.484290,1.484561,9.498889e-07,1.484292,1.484487,7.141026e-07,1.484205,1.484385,6.257632e-07,0.000136,0.000123,0.000014,48.562654,46.892862,42.914275,64.957265,78.514496,-56.410256,1.483897,1.485040,1.486183,0.153885,0.088668,0.999125,4,1,0.887885,0.460065,0.866025,0.5,1
2010-01-05 05:00:00,1.48420,1.48500,1.48390,1.48440,1.484633,1.484541,4.633333e-07,1.484980,1.484609,4.770000e-07,1.484320,1.484532,9.462222e-07,1.484308,1.484475,7.141026e-07,1.484150,1.484386,5.363158e-07,0.000117,0.000083,0.000034,50.583556,49.733909,47.930167,57.264957,68.334171,-48.717949,1.483745,1.484980,1.486215,0.166396,0.265273,1.000202,5,1,0.979084,0.203456,0.866025,0.5,1
2010-01-05 06:00:00,1.48450,1.48620,1.48440,1.48590,1.484800,1.485221,9.300000e-07,1.485020,1.485039,5.570000e-07,1.484630,1.484780,8.601111e-07,1.484423,1.484678,9.102564e-07,1.484240,1.484531,6.888421e-07,0.000220,0.000149,0.000072,59.212352,61.250402,65.904938,61.538462,61.253561,-10.256410,1.483685,1.485020,1.486355,0.179804,0.829572,1.001011,6,1,0.997669,-0.068242,0.866025,0.5,0
2010-01-05 07:00:00,1.48580,1.48620,1.48540,1.48560,1.485300,1.485410,6.300000e-07,1.485080,1.485226,6.170000e-07,1.484880,1.484929,6.351111e-07,1.484523,1.484810,1.013590e-06,1.484320,1.484632,7.764211e-07,0.000275,0.000163,0.000112,57.066144,58.283012,60.862525,74.358974,64.387464,-17.948718,1.483675,1.485080,1.486485,0.189233,0.685036,0.999798,7,1,0.942261,-0.334880,0.866025,0.5,0
2010-01-05 08:00:00,1.48560,1.48620,1.48460,1.48600,1.485833,1.485705,4.333333e-08,1.485200,1.485484,7.850000e-07,1.485110,1.485124,5.610000e-07,1.484669,1.484980,1.157308e-06,1.484445,1.484763,8.731316e-07,0.000347,0.000187,0.000159,59.190114,61.076662,65.131063,88.034188,74.643875,-7.692308,1.483615,1.485200,1.486785,0.213430,0.752377,1.000269,8,1,0.816970,-0.576680,0.866025,0.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-24 12:00:00,1.03934,1.04009,1.03900,1.03984,1.039567,1.039689,6.413334e-08,1.039744,1.039699,9.423000e-08,1.039896,1.039728,2.269600e-07,1.039716,1.039752,2.875256e-07,1.039673,1.039860,2.858871e-07,-0.000240,0.000069,-0.000309,48.571659,49.986673,52.466168,32.403101,38.036176,-54.883721,1.039195,1.039744,1.040293,0.105627,0.587412,1.000481,12,4,-0.136167,-0.990686,-0.866025,-0.5,None
2021-12-24 13:00:00,1.03984,1.03989,1.03947,1.03956,1.039580,1.039624,6.280001e-08,1.039670,1.039653,8.720000e-08,1.039776,1.039697,1.405600e-07,1.039753,1.039725,2.543564e-07,1.039665,1.039831,2.863211e-07,-0.000237,0.000058,-0.000295,46.311921,46.693946,46.388076,33.023256,33.901809,-67.906977,1.039142,1.039670,1.040198,0.101617,0.395881,0.999731,13,4,-0.398401,-0.917211,-0.866025,-0.5,None
2021-12-24 14:00:00,1.03955,1.04010,1.03944,1.03993,1.039777,1.039777,3.723334e-08,1.039638,1.039745,5.872000e-08,1.039746,1.039740,1.192933e-07,1.039818,1.039754,2.158026e-07,1.039625,1.039841,2.286368e-07,-0.000202,0.000074,-0.000276,49.645723,51.394885,54.708243,41.432227,35.619528,-52.912621,1.039205,1.039638,1.040071,0.083390,0.836810,1.000356,14,4,-0.631088,-0.775711,-0.866025,-0.5,None


In [14]:
df.reset_index(inplace=True)

In [15]:
logger.info(f' df shape before merge wiith 3 barriers additional info is {df.shape}')
df = df.merge(crypto.df_3_barriers_additional_info[['datetime', 'time_step']], on='datetime', how='inner')
logger.info(f' df shape after merge wiith 3 barriers additional info is {df.shape}')

2022-12-08 22:29:48,042 - root -  df shape before merge wiith 3 barriers additional info is (74478, 42)
2022-12-08 22:29:48,089 - root -  df shape after merge wiith 3 barriers additional info is (74478, 43)


In [16]:
df

,datetime,open,high,low,close,SMA_3,EMA_3,VAR_3,SMA_5,EMA_5,VAR_5,SMA_10,EMA_10,VAR_10,SMA_13,EMA_13,VAR_13,SMA_20,EMA_20,VAR_20,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,RSI_10,RSI_6,STOCHk_14_3_3,STOCHd_14_3_3,WILLR_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,roc_1,hour,weekday,hour_sin,hour_cos,weekday_sin,weekday_cos,y_pred,time_step
0,2010-01-05 04:00:00,1.48540,1.48560,1.48340,1.48410,1.484933,1.484683,5.233333e-07,1.485040,1.484714,4.080000e-07,1.484290,1.484561,9.498889e-07,1.484292,1.484487,7.141026e-07,1.484205,1.484385,6.257632e-07,0.000136,0.000123,0.000014,48.562654,46.892862,42.914275,64.957265,78.514496,-56.410256,1.483897,1.485040,1.486183,0.153885,0.088668,0.999125,4,1,0.887885,0.460065,0.866025,0.5,1,2
1,2010-01-05 05:00:00,1.48420,1.48500,1.48390,1.48440,1.484633,1.484541,4.633333e-07,1.484980,1.484609,4.770000e-07,1.484320,1.484532,9.462222e-07,1.484308,1.484475,7.141026e-07,1.484150,1.484386,5.363158e-07,0.000117,0.000083,0.000034,50.583556,49.733909,47.930167,57.264957,68.334171,-48.717949,1.483745,1.484980,1.486215,0.166396,0.265273,1.000202,5,1,0.979084,0.203456,0.866025,0.5,1,1
2,2010-01-05 06:00:00,1.48450,1.48620,1.48440,1.48590,1.484800,1.485221,9.300000e-07,1.485020,1.485039,5.570000e-07,1.484630,1.484780,8.601111e-07,1.484423,1.484678,9.102564e-07,1.484240,1.484531,6.888421e-07,0.000220,0.000149,0.000072,59.212352,61.250402,65.904938,61.538462,61.253561,-10.256410,1.483685,1.485020,1.486355,0.179804,0.829572,1.001011,6,1,0.997669,-0.068242,0.866025,0.5,0,11
3,2010-01-05 07:00:00,1.48580,1.48620,1.48540,1.48560,1.485300,1.485410,6.300000e-07,1.485080,1.485226,6.170000e-07,1.484880,1.484929,6.351111e-07,1.484523,1.484810,1.013590e-06,1.484320,1.484632,7.764211e-07,0.000275,0.000163,0.000112,57.066144,58.283012,60.862525,74.358974,64.387464,-17.948718,1.483675,1.485080,1.486485,0.189233,0.685036,0.999798,7,1,0.942261,-0.334880,0.866025,0.5,0,11
4,2010-01-05 08:00:00,1.48560,1.48620,1.48460,1.48600,1.485833,1.485705,4.333333e-08,1.485200,1.485484,7.850000e-07,1.485110,1.485124,5.610000e-07,1.484669,1.484980,1.157308e-06,1.484445,1.484763,8.731316e-07,0.000347,0.000187,0.000159,59.190114,61.076662,65.131063,88.034188,74.643875,-7.692308,1.483615,1.485200,1.486785,0.213430,0.752377,1.000269,8,1,0.816970,-0.576680,0.866025,0.5,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74473,2021-12-24 12:00:00,1.03934,1.04009,1.03900,1.03984,1.039567,1.039689,6.413334e-08,1.039744,1.039699,9.423000e-08,1.039896,1.039728,2.269600e-07,1.039716,1.039752,2.875256e-07,1.039673,1.039860,2.858871e-07,-0.000240,0.000069,-0.000309,48.571659,49.986673,52.466168,32.403101,38.036176,-54.883721,1.039195,1.039744,1.040293,0.105627,0.587412,1.000481,12,4,-0.136167,-0.990686,-0.866025,-0.5,None,None
74474,2021-12-24 13:00:00,1.03984,1.03989,1.03947,1.03956,1.039580,1.039624,6.280001e-08,1.039670,1.039653,8.720000e-08,1.039776,1.039697,1.405600e-07,1.039753,1.039725,2.543564e-07,1.039665,1.039831,2.863211e-07,-0.000237,0.000058,-0.000295,46.311921,46.693946,46.388076,33.023256,33.901809,-67.906977,1.039142,1.039670,1.040198,0.101617,0.395881,0.999731,13,4,-0.398401,-0.917211,-0.866025,-0.5,None,None
74475,2021-12-24 14:00:00,1.03955,1.04010,1.03944,1.03993,1.039777,1.039777,3.723334e-08,1.039638,1.039745,5.872000e-08,1.039746,1.039740,1.192933e-07,1.039818,1.039754,2.158026e-07,1.039625,1.039841,2.286368e-07,-0.000202,0.000074,-0.000276,49.645723,51.394885,54.708243,41.432227,35.619528,-52.912621,1.039205,1.039638,1.040071,0.083390,0.836810,1.000356,14,4,-0.631088,-0.775711,-0.866025,-0.5,None,None
74476,2021-12-24 15:00:00,1.03993,1.04042,1.03940,1.04036,1.039950,1.040069,1.603000e-07,1.039806,1.039950,1.502800e-07,1.039747,1.039853,1.206456e-07,1.039908,1.039841,1.974974e-07,1.039689,1.039890,2.371253e-07,-0.000138,0.000110,-0.000249,53.276882,56.363983,62.766673,49.047189,41.167557,-

In [17]:
logger.info(f' df shape before droping na {df.shape}')
df.dropna(inplace=True)
logger.info(f' df shape after droping {df.shape}')

2022-12-08 22:29:50,622 - root -  df shape before droping na (74478, 43)
2022-12-08 22:29:50,703 - root -  df shape after droping (74467, 43)


In [18]:
df.set_index('datetime', inplace=True)
df.head()

,open,high,low,close,SMA_3,EMA_3,VAR_3,SMA_5,EMA_5,VAR_5,SMA_10,EMA_10,VAR_10,SMA_13,EMA_13,VAR_13,SMA_20,EMA_20,VAR_20,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,RSI_10,RSI_6,STOCHk_14_3_3,STOCHd_14_3_3,WILLR_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,roc_1,hour,weekday,hour_sin,hour_cos,weekday_sin,weekday_cos,y_pred,time_step
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-05 04:00:00,1.4854,1.4856,1.4834,1.4841,1.484933,1.484683,5.233333e-07,1.48504,1.484714,4.080000e-07,1.48429,1.484561,9.498889e-07,1.484292,1.484487,7.141026e-07,1.484205,1.484385,6.257632e-07,0.000136,0.000123,0.000014,48.562654,46.892862,42.914275,64.957265,78.514496,-56.410256,1.483897,1.48504,1.486183,0.153885,0.088668,0.999125,4,1,0.887885,0.460065,0.866025,0.5,1,2
2010-01-05 05:00:00,1.4842,1.4850,1.4839,1.4844,1.484633,1.484541,4.633333e-07,1.48498,1.484609,4.770000e-07,1.48432,1.484532,9.462222e-07,1.484308,1.484475,7.141026e-07,1.484150,1.484386,5.363158e-07,0.000117,0.000083,0.000034,50.583556,49.733909,47.930167,57.264957,68.334171,-48.717949,1.483745,1.48498,1.486215,0.166396,0.265273,1.000202,5,1,0.979084,0.203456,0.866025,0.5,1,1
2010-01-05 06:00:00,1.4845,1.4862,1.4844,1.4859,1.484800,1.485221,9.300000e-07,1.48502,1.485039,5.570000e-07,1.48463,1.484780,8.601111e-07,1.484423,1.484678,9.102564e-07,1.484240,1.484531,6.888421e-07,0.000220,0.000149,0.000072,59.212352,61.250402,65.904938,61.538462,61.253561,-10.256410,1.483685,1.48502,1.486355,0.179804,0.829572,1.001011,6,1,0.997669,-0.068242,0.866025,0.5,0,11
2010-01-05 07:00:00,1.4858,1.4862,1.4854,1.4856,1.485300,1.485410,6.300000e-07,1.48508,1.485226,6.170000e-07,1.48488,1.484929,6.351111e-07,1.484523,1.484810,1.013590e-06,1.484320,1.484632,7.764211e-07,0.000275,0.000163,0.000112,57.066144,58.283012,60.862525,74.358974,64.387464,-17.948718,1.483675,1.48508,1.486485,0.189233,0.685036,0.999798,7,1,0.942261,-0.334880,0.866025,0.5,0,11
2010-01-05 08:00:00,1.4856,1.4862,1.4846,1.4860,1.485833,1.485705,4.333333e-08,1.48520,1.485484,7.850000e-07,1.48511,1.485124,5.610000e-07,1.484669,1.484980,1.157308e-06,1.484445,1.484763,8.731316e-07,0.000347,0.000187,0.000159,59.190114,61.076662,65.131063,88.034188,74.643875,-7.692308,1.483615,1.48520,1.486785,0.213430,0.752377,1.000269,8,1,0.816970,-0.576680,0.866025,0.5,0,11


In [19]:
df.shape

(74467, 42)

In [29]:
X, y, X_val, y_val, X_test, y_test, y_cat, y_val_cat, y_test_cat, _ = train_test_val_split(df, 
                         nb_classes=config.nb_classes, freq=config.freq,
                         seq_len=config.seq_len, steps_ahead=config.steps_ahead,
                         train_end=config.train_end, val_end=config.val_end,
                         test_end=config.test_end, binarize_target=False, time_step=True)

ValueError: cannot insert level_0, already exists

In [ ]:
X.shape

(14326, 24, 42)

In [22]:
mlflow.tensorflow.autolog()
mlflow.set_experiment(experiment_name=get_ml_flow_experiment_name())
callbacks = get_callbacks()
model = model_factory(X)
history = model.fit(X, y_cat, batch_size=config.batch, epochs=10, verbose=2,
                    validation_data=(X_val, y_val_cat), callbacks=callbacks)
model.save(get_final_model_path())

2022/12/08 22:30:13 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.
2022-12-08 22:30:13.639807: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022/12/08 22:30:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c0ede174e31e46a8938ab82be6c5489c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow
2022-12-08 22:30:14.098402: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-12-08 22:30:14.098423: I tenso

Epoch 1/10


2022-12-08 22:30:17.418332: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-12-08 22:30:17.418358: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-12-08 22:30:17.447684: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-12-08 22:30:17.455658: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-12-08 22:30:17.464536: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /var/folders/fy/kbq5nlv945s7pr1wvh78h5840000gn/T/tmp3j6tm8bi/train/plugins/profile/2022_12_08_22_30_17

2022-12-08 22:30:17.469010: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to /var/folders/fy/kbq5nlv945s7pr1wvh78h5840000gn/T/tmp3j6tm8bi/train/plugins/profile/2022_12_08_22_30_17/macC02X74LNJG5J.trace.json.gz
2022-12-08 22:30:17.489791: I tensorflow/core/profiler/rpc/client/sav

971/971 - 32s - loss: 0.6947 - accuracy: 0.5389 - val_loss: 0.6885 - val_accuracy: 0.5439

Epoch 00001: val_accuracy improved from -inf to 0.54391, saving model to /Users/i0495036/Documents/sandbox/ml_investing_wne/ml_investing_wne/src/ml_investing_wne/models/resnet_hist_data_EURCHF_60min_1.h5
Epoch 2/10
971/971 - 27s - loss: 0.6808 - accuracy: 0.5625 - val_loss: 0.6869 - val_accuracy: 0.5393

Epoch 00002: val_accuracy did not improve from 0.54391
Epoch 3/10
971/971 - 37s - loss: 0.6739 - accuracy: 0.5752 - val_loss: 0.6881 - val_accuracy: 0.5380

Epoch 00003: val_accuracy did not improve from 0.54391
Epoch 4/10
971/971 - 30s - loss: 0.6663 - accuracy: 0.5897 - val_loss: 0.6920 - val_accuracy: 0.5439

Epoch 00004: val_accuracy did not improve from 0.54391
Epoch 5/10
971/971 - 39s - loss: 0.6585 - accuracy: 0.6013 - val_loss: 0.6918 - val_accuracy: 0.5466

Epoch 00005: val_accuracy improved from 0.54391 to 0.54664, saving model to /Users/i0495036/Documents/sandbox/ml_investing_wne/ml_in

2022-12-08 22:35:53.550612: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/fy/kbq5nlv945s7pr1wvh78h5840000gn/T/tmp9liuhisu/model/data/model/assets


2022-12-08 22:35:58,894 - tensorflow - Assets written to: /var/folders/fy/kbq5nlv945s7pr1wvh78h5840000gn/T/tmp9liuhisu/model/data/model/assets
/Users/i0495036/Documents/sandbox/ml_investing_wne/ml_investing_wne/venv/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: /Users/i0495036/Documents/sandbox/ml_investing_wne/ml_investing_wne/src/ml_investing_wne/models/production/resnet_hist_data_EURCHF_60min_1_24/assets


2022-12-08 22:36:18,244 - tensorflow - Assets written to: /Users/i0495036/Documents/sandbox/ml_investing_wne/ml_investing_wne/src/ml_investing_wne/models/production/resnet_hist_data_EURCHF_60min_1_24/assets
/Users/i0495036/Documents/sandbox/ml_investing_wne/ml_investing_wne/venv/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [23]:
df['cost'] = (config.pips / 10000) / df['close']

In [24]:
import joblib
def load_test_dates():

    name = f'test_{config.currency}_{config.freq}.save'

    start_date = joblib.load(os.path.join(config.package_directory, 'models',
                                        f'first_sequence_ends_{name}'))
    end_date = joblib.load(os.path.join(config.package_directory, 'models',
                                        f'last_sequence_ends_{name}'))

    return start_date, end_date

In [25]:
test_loss, test_acc = model.evaluate(X_test, y_test_cat)
logger.info('Test accuracy : %.4f', test_acc)
logger.info('Test loss : %.4f', test_loss)
mlflow.log_metric("test_acc", test_acc)
mlflow.log_metric("test_loss", test_loss)
mlflow.log_metric("test_loss", test_loss)
mlflow.set_tag('currency', config.currency)
mlflow.set_tag('frequency', config.freq)
mlflow.set_tag('steps_ahead', config.steps_ahead)
mlflow.log_metric('y_distribution', y.mean())
mlflow.log_metric('y_val_distribution', y_val.mean())
mlflow.log_metric('y_test_distribution', y_test.mean())
mlflow.log_metric('cost', config.pips)
mlflow.log_metric('seq_len', config.seq_len)
y_pred = model.predict(X_test)
y_pred_class = y_pred.argmax(axis=-1)
start_date, end_date = load_test_dates()
lower_bounds = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
upper_bounds = [1 - lower for lower in lower_bounds]

192/192 [==============================] - 1s 4ms/step - loss: 0.8123 - accuracy: 0.5266


2022-12-08 22:36:30,955 - root - Test accuracy : 0.5266
2022-12-08 22:36:30,956 - root - Test loss : 0.8123


In [27]:
df.head()

,datetime,open,high,low,close,SMA_3,EMA_3,VAR_3,SMA_5,EMA_5,VAR_5,SMA_10,EMA_10,VAR_10,SMA_13,EMA_13,VAR_13,SMA_20,EMA_20,VAR_20,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,RSI_10,RSI_6,STOCHk_14_3_3,STOCHd_14_3_3,WILLR_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,roc_1,hour,weekday,hour_sin,hour_cos,weekday_sin,weekday_cos,y_pred,time_step,cost
0,2010-01-05 04:00:00,1.4854,1.4856,1.4834,1.4841,1.484933,1.484683,5.233333e-07,1.48504,1.484714,4.080000e-07,1.48429,1.484561,9.498889e-07,1.484292,1.484487,7.141026e-07,1.484205,1.484385,6.257632e-07,0.000136,0.000123,0.000014,48.562654,46.892862,42.914275,64.957265,78.514496,-56.410256,1.483897,1.48504,1.486183,0.153885,0.088668,0.999125,4,1,0.887885,0.460065,0.866025,0.5,1,2,0.000135
1,2010-01-05 05:00:00,1.4842,1.4850,1.4839,1.4844,1.484633,1.484541,4.633333e-07,1.48498,1.484609,4.770000e-07,1.48432,1.484532,9.462222e-07,1.484308,1.484475,7.141026e-07,1.484150,1.484386,5.363158e-07,0.000117,0.000083,0.000034,50.583556,49.733909,47.930167,57.264957,68.334171,-48.717949,1.483745,1.48498,1.486215,0.166396,0.265273,1.000202,5,1,0.979084,0.203456,0.866025,0.5,1,1,0.000135
2,2010-01-05 06:00:00,1.4845,1.4862,1.4844,1.4859,1.484800,1.485221,9.300000e-07,1.48502,1.485039,5.570000e-07,1.48463,1.484780,8.601111e-07,1.484423,1.484678,9.102564e-07,1.484240,1.484531,6.888421e-07,0.000220,0.000149,0.000072,59.212352,61.250402,65.904938,61.538462,61.253561,-10.256410,1.483685,1.48502,1.486355,0.179804,0.829572,1.001011,6,1,0.997669,-0.068242,0.866025,0.5,0,11,0.000135
3,2010-01-05 07:00:00,1.4858,1.4862,1.4854,1.4856,1.485300,1.485410,6.300000e-07,1.48508,1.485226,6.170000e-07,1.48488,1.484929,6.351111e-07,1.484523,1.484810,1.013590e-06,1.484320,1.484632,7.764211e-07,0.000275,0.000163,0.000112,57.066144,58.283012,60.862525,74.358974,64.387464,-17.948718,1.483675,1.48508,1.486485,0.189233,0.685036,0.999798,7,1,0.942261,-0.334880,0.866025,0.5,0,11,0.000135
4,2010-01-05 08:00:00,1.4856,1.4862,1.4846,1.4860,1.485833,1.485705,4.333333e-08,1.48520,1.485484,7.850000e-07,1.48511,1.485124,5.610000e-07,1.484669,1.484980,1.157308e-06,1.484445,1.484763,8.731316e-07,0.000347,0.000187,0.000159,59.190114,61.076662,65.131063,88.034188,74.643875,-7.692308,1.483615,1.48520,1.486785,0.213430,0.752377,1.000269,8,1,0.816970,-0.576680,0.866025,0.5,0,11,0.000135


In [18]:
upper_bound = 0.6
lower_bound = 0.4


,index,datetime,open,high,low,close,y_pred,SMA_3,EMA_3,VAR_3,...,BBB_5_2.0,BBP_5_2.0,roc_1,hour,weekday,hour_sin,hour_cos,weekday_sin,weekday_cos,cost
0,0,2010-01-24 12:00:00,1.47240,1.47400,1.47140,1.47150,0,1.472133,1.471879,7.233333e-07,...,0.280094,0.519411,0.998914,12,6,-2.449294e-16,1.0,-2.449294e-16,1.0,0.000136
1,1,2010-01-25 00:00:00,1.47160,1.47330,1.47120,1.47150,0,1.472033,1.471689,8.533333e-07,...,0.251126,0.467529,1.000000,0,0,0.000000e+00,1.0,0.000000e+00,1.0,0.000136
2,2,2010-01-25 12:00:00,1.47140,1.47200,1.47040,1.47120,1,1.471400,1.471445,3.000000e-08,...,0.181417,0.267802,0.999796,12,0,-2.449294e-16,1.0,0.000000e+00,1.0,0.000136
3,3,2010-01-26 00:00:00,1.47120,1.47400,1.47040,1.47150,1,1.471400,1.471472,3.000000e-08,...,0.184813,0.404412,1.000204,0,1,0.000000e+00,1.0,8.660254e-01,0.5,0.000136
4,4,2010-01-26 12:00:00,1.47140,1.47410,1.47130,1.47190,0,1.471533,1.471686,1.233333e-07,...,0.060539,0.926563,1.000272,12,1,-2.449294e-16,1.0,8.660254e-01,0.5,0.000136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6808,6808,2021-12-22 00:00:00,1.04176,1.04341,1.04157,1.04194,1,1.041787,1.041558,2.013333e-08,...,0.431979,0.736635,1.000173,0,2,0.000000e+00,1.0,8.660254e-01,-0.5,0.000192
6809,6809,2021-12-22 12:00:00,1.04194,1.04267,1.04125,1.04209,0,1.041930,1.041824,2.730000e-08,...,0.340083,0.687480,1.000144,12,2,-2.449294e-16,1.0,8.660254e-01,-0.5,0.000192
6810,6810,2021-12-23 00:00:00,1.04209,1.04320,1.03913,1.03987,0,1.041300,1.040847,1.539300e-06,...,0.311329,0.008386,0.997870,0,3,0.000000e+00,1.0,1.224647e-16,-1.0,0.000192
6811,6811,2021-12-23 12:00:00,1.03987,1.04081,1.03887,1.03939,0,1.040450,1.040118,2.074800e-06,...,0.438703,0.145277,0.999538,12,3,-2.449294e-16,1.0,1.224647e-16,-1.0,0.000192


In [19]:

# recreate target as continous variable
df['y_pred'] = df['close'].shift(-config.steps_ahead) / df['close'] - 1
prediction

,index,datetime,open,high,low,close,y_pred,SMA_3,EMA_3,VAR_3,...,BBB_5_2.0,BBP_5_2.0,roc_1,hour,weekday,hour_sin,hour_cos,weekday_sin,weekday_cos,cost
0,0,2010-01-24 12:00:00,1.47240,1.47400,1.47140,1.47150,0,1.472133,1.471879,7.233333e-07,...,0.280094,0.519411,0.998914,12,6,-2.449294e-16,1.0,-2.449294e-16,1.0,0.000136
1,1,2010-01-25 00:00:00,1.47160,1.47330,1.47120,1.47150,0,1.472033,1.471689,8.533333e-07,...,0.251126,0.467529,1.000000,0,0,0.000000e+00,1.0,0.000000e+00,1.0,0.000136
2,2,2010-01-25 12:00:00,1.47140,1.47200,1.47040,1.47120,1,1.471400,1.471445,3.000000e-08,...,0.181417,0.267802,0.999796,12,0,-2.449294e-16,1.0,0.000000e+00,1.0,0.000136
3,3,2010-01-26 00:00:00,1.47120,1.47400,1.47040,1.47150,1,1.471400,1.471472,3.000000e-08,...,0.184813,0.404412,1.000204,0,1,0.000000e+00,1.0,8.660254e-01,0.5,0.000136
4,4,2010-01-26 12:00:00,1.47140,1.47410,1.47130,1.47190,0,1.471533,1.471686,1.233333e-07,...,0.060539,0.926563,1.000272,12,1,-2.449294e-16,1.0,8.660254e-01,0.5,0.000136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6808,6808,2021-12-22 00:00:00,1.04176,1.04341,1.04157,1.04194,1,1.041787,1.041558,2.013333e-08,...,0.431979,0.736635,1.000173,0,2,0.000000e+00,1.0,8.660254e-01,-0.5,0.000192
6809,6809,2021-12-22 12:00:00,1.04194,1.04267,1.04125,1.04209,0,1.041930,1.041824,2.730000e-08,...,0.340083,0.687480,1.000144,12,2,-2.449294e-16,1.0,8.660254e-01,-0.5,0.000192
6810,6810,2021-12-23 00:00:00,1.04209,1.04320,1.03913,1.03987,0,1.041300,1.040847,1.539300e-06,...,0.311329,0.008386,0.997870,0,3,0.000000e+00,1.0,1.224647e-16,-1.0,0.000192
6811,6811,2021-12-23 12:00:00,1.03987,1.04081,1.03887,1.03939,0,1.040450,1.040118,2.074800e-06,...,0.438703,0.145277,0.999538,12,3,-2.449294e-16,1.0,1.224647e-16,-1.0,0.000192


In [11]:

# new_start = config.val_end + config.seq_len * datetime.timedelta(minutes=int(''.join(filter(str.isdigit, config.freq))))
prediction = df.loc[(df.datetime >= start_date) & (df.datetime <= end_date)]
if config.provider == 'hist_data':
    prediction['datetime_local'] = prediction['datetime'].dt.tz_localize('US/Eastern').dt.tz_convert(
        'Europe/London').dt.tz_localize(None)
else:
    prediction['datetime_local'] = prediction['datetime']
prediction['hour_local'] = prediction['datetime_local'].dt.time
prediction['prediction'] = y_pred[:, 1]
conditions = [
    (prediction['prediction'] <= lower_bound),
    (prediction['prediction'] > lower_bound) & (prediction['prediction'] <= upper_bound),
    (prediction['prediction'] > upper_bound)
]
values = [0, 0.5, 1]
prediction['trade'] = np.select(conditions, values)

prediction.reset_index(inplace=True)

In [12]:
prediction

,index,datetime,open,high,low,close,y_pred,SMA_3,EMA_3,VAR_3,...,weekday,hour_sin,hour_cos,weekday_sin,weekday_cos,cost,datetime_local,hour_local,prediction,trade
0,6540,2021-07-05 00:00:00,1.09334,1.09440,1.09312,1.09388,-0.000503,1.093083,1.093643,9.182333e-07,...,0,0.000000e+00,1.0,0.000000e+00,1.0,0.000183,2021-07-05 05:00:00,05:00:00,0.577751,0.5
1,6541,2021-07-05 12:00:00,1.09388,1.09457,1.09291,1.09333,-0.000951,1.093520,1.093486,9.730000e-08,...,0,-2.449294e-16,1.0,0.000000e+00,1.0,0.000183,2021-07-05 17:00:00,17:00:00,0.428526,0.5
2,6542,2021-07-06 00:00:00,1.09332,1.09451,1.09216,1.09229,0.000064,1.093167,1.092888,6.520333e-07,...,1,0.000000e+00,1.0,8.660254e-01,0.5,0.000183,2021-07-06 05:00:00,05:00:00,0.547930,0.5
3,6543,2021-07-06 12:00:00,1.09230,1.09345,1.09186,1.09236,-0.000302,1.092660,1.092624,3.379000e-07,...,1,-2.449294e-16,1.0,8.660254e-01,0.5,0.000183,2021-07-06 17:00:00,17:00:00,0.456381,0.5
4,6544,2021-07-07 00:00:00,1.09235,1.09264,1.09103,1.09203,-0.000650,1.092227,1.092327,3.023333e-08,...,2,0.000000e+00,1.0,8.660254e-01,-0.5,0.000183,2021-07-07 05:00:00,05:00:00,0.518932,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,6763,2021-11-23 12:00:00,1.05057,1.05058,1.04854,1.04934,-0.002030,1.049207,1.049231,2.058233e-06,...,1,-2.449294e-16,1.0,8.660254e-01,0.5,0.000191,2021-11-23 17:00:00,17:00:00,0.460120,0.5
224,6764,2021-11-24 00:00:00,1.04932,1.05017,1.04696,1.04721,-0.000191,1.049040,1.048221,2.889900e-06,...,2,0.000000e+00,1.0,8.660254e-01,-0.5,0.000191,2021-11-24 05:00:00,05:00:00,0.499349,0.5
225,6765,2021-11-24 12:00:00,1.04721,1.04738,1.04541,1.04701,0.001824,1.047853,1.047615,1.667633e-06,...,2,-2.449294e-16,1.0,8.660254e-01,-0.5,0.000191,2021-11-24 17:00:00,17:00:00,0.479965,0.5
226,6766,2021-11-25 00:00:00,1.04701,1.04934,1.04576,1.04892,-0.002069,1.047713,1.048268,1.102033e-06,...,3,0.000000e+00,1.0,1.224647e-16,-1.0,0.000191,2021-11-25 05:00:00,05:00:00,0.477776,0.5
